In [2]:
import os
import pandas as pd

Loading data from the folder

In [17]:
# Setting up path to load file
current_dir = os.getcwd()
csv_filename = 'data.csv'  
csv_path = os.path.join(current_dir, csv_filename)

# Initialize counter
bad_line_counter = 0

# Define a function to handle bad lines
def handle_bad_line(bad_line):
    global bad_line_counter
    bad_line_counter += 1
    return None  # Skip the bad line

# Read it in chunks due to huge size
chunk_size = 1_500_000

# Keep only relevant columns, remove ones with lots of NaN values
# LSOA name removed due to redundance
# Last outcome category removed due to high number of NaN values: 87 482 715 / 149 351 274 
# Context removed due to high number of NaN values: 144 750 473 / 149 351 274 
# Crime ID removed due to high number of NaN values: 5 781 005 / 27 205 384 (London)

columns_to_keep = ['Month', 'Falls within', 'Reported by', 'Latitude', 'Longitude', 'Location', 'LSOA code', 'Crime type']
chunks = pd.read_csv(csv_path, 
                     chunksize=chunk_size,
                     usecols=columns_to_keep,
                     engine='python', 
                     on_bad_lines=handle_bad_line)

dfs = []
for i, chunk in enumerate(chunks):

    # Removing NaN values from certain columns
    # Coordinates because it is not possible to find out then if it is in london or not
    # LSOA because it is only 6 values from London, hard to later assign police officers
    chunk = chunk.dropna(subset=['Latitude', 'Longitude', 'LSOA code'])

    # Keep only rows inside London's coordinated obtained from Wikipedia
    lat_min, lat_max = 51.2867602, 51.6918741
    lon_min, lon_max = -0.5103751, 0.3340155

    chunk = chunk[
        (chunk['Latitude'] >= lat_min) & (chunk['Latitude'] <= lat_max) &
        (chunk['Longitude'] >= lon_min) & (chunk['Longitude'] <= lon_max)
    ]
    # Printing progress
    print(f"Processing chunk {i+1}...")
    dfs.append(chunk)

df_london = pd.concat(dfs, ignore_index=True)

print(f"Loaded {len(df_london)} good rows.")
print(f"Skipped {bad_line_counter} bad rows.")

Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...
Processing chunk 19...
Processing chunk 20...
Processing chunk 21...
Processing chunk 22...
Processing chunk 23...
Processing chunk 24...
Processing chunk 25...
Processing chunk 26...
Processing chunk 27...
Processing chunk 28...
Processing chunk 29...
Processing chunk 30...
Processing chunk 31...
Processing chunk 32...
Processing chunk 33...
Processing chunk 34...
Processing chunk 35...
Processing chunk 36...
Processing chunk 37...
Processing chunk 38...
Processing chunk 39...
Processing chunk 40...
Processing chunk 41...
Processing chunk 42...
Processing chunk 43...
Processing chunk 44.

Printing head to see the Data Frame

In [18]:
df_london.head()

,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,Crime type
0,2010-12,British Transport Police,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Anti-social behaviour
1,2010-12,British Transport Police,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Anti-social behaviour
2,2010-12,British Transport Police,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Violent crime
3,2010-12,British Transport Police,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Violent crime
4,2010-12,British Transport Police,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Violent crime


Counting NaN values 

In [19]:
def count_nans(df):
    """
    Count NaN values in each column of a df object
    Input:
    - df: df object obtained from data
    Returns:
    - dictionary with numbers of NaN values for each column
    """
    nan_counts = df.isna().sum().to_dict()
    return nan_counts

count_nans(df_london)

{'Month': 0,
 'Reported by': 0,
 'Falls within': 0,
 'Longitude': 0,
 'Latitude': 0,
 'Location': 0,
 'LSOA code': 0,
 'Crime type': 0}

Data exploration space

In [20]:
# Checking certain column's unique values
column_name = 'Falls within'
counts = df_london[column_name].value_counts(dropna=False).to_dict()
len(counts)

37

In [22]:
# Check length
len(df_london)

27205378

In [23]:
# Check where 'Reported by' and 'Falls within' are different
different_rows = df_london[df_london['Reported by'] != df_london['Falls within']]

# Print how number of different rows
len(different_rows)

492

In [25]:
# Select rows where 'Reported by' and 'Falls within' are different
different_rows = df_london[df_london['Reported by'] != df_london['Falls within']]

# Create a dictionary: {Reported by : Falls within}
diff_dict = dict(zip(different_rows['Reported by'], different_rows['Falls within']))

# Print results
print(diff_dict)


{'Surrey Police': 'City of London Police'}


In [26]:
# Removing Reported by column due to redundancy, every value that was 'Surrey Police' will now be 'City of London Police'
df_london = df_london.drop(columns=['Reported by'])
cleaned_london_data = df_london.rename(columns={'Falls within': 'Falls within/Reported by'})

Converting df into csv file for storage

In [27]:
cleaned_london_data.to_csv('london_data.csv', index=False)

KeyboardInterrupt: 

SKIP HERE FOR ANALYSIS AND EXPLORATION OF CLEANED FILE

Loading new csv file and getting it ready for future work

In [28]:
import os
import pandas as pd

In [29]:
# Get directory and path for the csv file
current_dir2 = os.getcwd()
csv_filename2 = 'london_data.csv'  
csv_path2 = os.path.join(current_dir2, csv_filename2)

df = pd.read_csv(csv_path2)
df.head()

,Month,Falls within/Reported by,Longitude,Latitude,Location,LSOA code,Crime type
0,2010-12,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Anti-social behaviour
1,2010-12,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Anti-social behaviour
2,2010-12,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Violent crime
3,2010-12,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Violent crime
4,2010-12,British Transport Police,0.165776,51.5442,On or near Dagenham East,E01000036,Violent crime


All crimes and their amounts that happened in London: 

In [30]:
crime_counts = df_london['Crime type'].value_counts()
print(crime_counts)


Crime type
Investigation complete; no suspect identified          7147189
Anti-social behaviour                                  4473527
Violence and sexual offences                           2976924
Other theft                                            1813567
Vehicle crime                                          1564603
Burglary                                               1138550
Criminal damage and arson                               909213
Suspect charged                                         902310
Shoplifting                                             719539
Public order                                            696050
Theft from the person                                   651241
Other crime                                             626423
Unable to prosecute suspect                             607665
Drugs                                                   602790
Robbery                                                 445325
Violent crime                               

The percentage of the crime within all the crimes:

In [31]:
crime_percentages = (crime_counts / crime_counts.sum()) * 100
print(crime_percentages)


Crime type
Investigation complete; no suspect identified          26.271236
Anti-social behaviour                                  16.443539
Violence and sexual offences                           10.942410
Other theft                                             6.666208
Vehicle crime                                           5.751080
Burglary                                                4.185018
Criminal damage and arson                               3.342034
Suspect charged                                         3.316660
Shoplifting                                             2.644841
Public order                                            2.558501
Theft from the person                                   2.393795
Other crime                                             2.302570
Unable to prosecute suspect                             2.233621
Drugs                                                   2.215702
Robbery                                                 1.636901
Violent crime 

The trends over time:

In [33]:
df_london['Month'] = pd.to_datetime(df_london['Month'], errors='coerce')
trend = df_london.groupby([df_london['Month'].dt.to_period('M'), 'Crime type']).size().unstack().fillna(0)
print(trend)

Crime type  Action to be taken by another organisation  Anti-social behaviour  \
Month                                                                           
2010-12                                            0.0                77430.0   
2011-01                                            0.0                74927.0   
2011-02                                            0.0                35938.0   
2011-03                                            0.0                40102.0   
2011-04                                            0.0                45614.0   
...                                                ...                    ...   
2024-10                                          759.0                21726.0   
2024-11                                         1100.0                19417.0   
2024-12                                          713.0                16837.0   
2025-01                                          637.0                17038.0   
2025-02                     